In [1]:
import warnings
warnings.filterwarnings("ignore")
from ta.trend import MACD, EMAIndicator
from ta.momentum import RSIIndicator

last_trade = None  # Track the last trade type (either "call" or "put")

def calculate_fractals(high, low, window=3):
    bullish_fractals = []
    bearish_fractals = []
    
    for i in range(window, len(high)-window):
        # Bearish Fractal (High)
        if all(high[i] > high[j] for j in range(i-window, i)) and \
           all(high[i] > high[j] for j in range(i+1, i+window+1)):
            bearish_fractals.append(True)
        else:
            bearish_fractals.append(False)
            
        # Bullish Fractal (Low)
        if all(low[i] < low[j] for j in range(i-window, i)) and \
           all(low[i] < low[j] for j in range(i+1, i+window+1)):
            bullish_fractals.append(True)
        else:
            bullish_fractals.append(False)
            
    return bullish_fractals, bearish_fractals



In [2]:
from trading_tools import data_frame


ASSET = "AUDCAD_otc" 
df = data_frame.load_csv(f"actives/ACTIVO-{ASSET}-0060s.csv")
asserts = 0
total_trades = 0
for index in range(len(df - 34)):
        start = index
        end = index + 34
        data = df.iloc[index:index + 34]
        data_close = data.iloc[-1]["close"]
        target_close = df.iloc[end]["close"]
        # print(len(data), data.iloc[-1]["close"], df.iloc[end]["close"], df.iloc[end - 1]["close"])
        # MACD calculation
        macd_indicator = MACD(data["close"], window_slow=26, window_fast=12, window_sign=9)
        macd_line = macd_indicator.macd()
        signal_line = macd_indicator.macd_signal()
        histogram = macd_indicator.macd_diff()

        # RSI calculation
        rsi = RSIIndicator(data["close"], window=14).rsi()
        
        # EMA calculation
        ema_25 = EMAIndicator(data["close"], window=25).ema_indicator()

        # Cálculo de fractales
        bullish_fractals, bearish_fractals = calculate_fractals(data["high"], data["low"])

        # Get the last values
        macd_val = macd_line.iloc[-1]
        signal_val = signal_line.iloc[-1]
        hist_val = histogram.iloc[-1]
        rsi_val = rsi.iloc[-1]
        ema_val = ema_25.iloc[-1]
        
        # Análisis de tendencia usando MACD y EMA
        trend_up = (histogram.iloc[-3:].mean() > 0 and 
                   data["close"].iloc[-1] > ema_val and 
                   rsi_val > 50)  # Tendencia alcista
        
        trend_down = (histogram.iloc[-3:].mean() < 0 and 
                     data["close"].iloc[-1] < ema_val and 
                     rsi_val < 50)  # Tendencia bajista

        # Condiciones de trading
        bullish_crossover = (macd_val > signal_val and 
                           trend_up and 
                           (len(bullish_fractals) > 0 and bullish_fractals[-1]))
        
        bearish_crossover = (macd_val < signal_val and 
                           trend_down and 
                           (len(bearish_fractals) > 0 and bearish_fractals[-1]))
        
        # print(bullish_crossover, bearish_crossover )
        
        if bullish_crossover and last_trade != "call":
            total_trades += 1
            # print("Bullish crossover with trend - Placing a CALL order")
            # print(f"RSI: {rsi_val:.2f} | MACD Hist: {hist_val:.5f}")
            # api.Call(expiration=60, amount=1)
            if target_close > data_close:
                asserts += 1
            print(f"{float(asserts) / float(total_trades) * 100:.2f}%")
            
            last_trade = "call"

        elif bearish_crossover and last_trade != "put":
            total_trades += 1
            # print("Bearish crossover with trend - Placing a PUT order")
            # print(f"RSI: {rsi_val:.2f} | MACD Hist: {hist_val:.5f}")
            # api.Put(expiration=60, amount=1)
            if target_close < data_close:
                asserts += 1
            print(f"{asserts} / {total_trades}  -- accuracy  {float(asserts) / float(total_trades) * 100:.2f}%")
            last_trade = "put"
        # else:
        #     print("No trade... Waiting for better conditions", end='\r')


0 / 1  -- accuracy  0.00%
0.00%
1 / 3  -- accuracy  33.33%
50.00%
3 / 5  -- accuracy  60.00%
50.00%
3 / 7  -- accuracy  42.86%
37.50%
3 / 9  -- accuracy  33.33%
30.00%
4 / 11  -- accuracy  36.36%
41.67%
6 / 13  -- accuracy  46.15%
50.00%
8 / 15  -- accuracy  53.33%
50.00%
8 / 17  -- accuracy  47.06%
44.44%
8 / 19  -- accuracy  42.11%
40.00%
9 / 21  -- accuracy  42.86%
45.45%
10 / 23  -- accuracy  43.48%
45.83%
12 / 25  -- accuracy  48.00%
50.00%
14 / 27  -- accuracy  51.85%
50.00%
15 / 29  -- accuracy  51.72%
50.00%
15 / 31  -- accuracy  48.39%
50.00%
17 / 33  -- accuracy  51.52%
50.00%
17 / 35  -- accuracy  48.57%
50.00%
18 / 37  -- accuracy  48.65%
47.37%
18 / 39  -- accuracy  46.15%
45.00%
19 / 41  -- accuracy  46.34%
47.62%
20 / 43  -- accuracy  46.51%
47.73%
21 / 45  -- accuracy  46.67%
47.83%
23 / 47  -- accuracy  48.94%
47.92%
23 / 49  -- accuracy  46.94%
46.00%
23 / 51  -- accuracy  45.10%
44.23%
24 / 53  -- accuracy  45.28%
46.30%
25 / 55  -- accuracy  45.45%
44.64%
26 / 57  -

IndexError: single positional indexer is out-of-bounds